In [ ]:
# Below install will be necessary in every notebook on AWS

In [ ]:
# Need script to convert to numeric for everything to input into model

In [1]:
!pip install transformers torch scikit-learn boto3 sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 37.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 89.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 156.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 166.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 45.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 130.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 158.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 137.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 133.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.

In [2]:
import argparse
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import os
import numpy as np
import sagemaker
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.image_uris import retrieve
import boto3

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/01/25 19:55:42] INFO     Found credentials from IAM Role:                                   ]8;id=193827;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=823329;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Setup S3 paths, chunk size, etc.

role = "arn:aws:iam::221082214706:role/MYLabRole" # Hardcode, maybe avoids errors
region = "us-east-1"

# Boto3 + SageMaker session 
boto_sess = boto3.Session(region_name=region)
session = sagemaker.Session(boto_session=boto_sess)
print(f"Using region: {region}")
print(f"USing role: {role}")

bucket_name = "arxiv-project-bucket"
prefix_in = "processed-data/"
prefix_out = "processed-data/embeddings/"

[02/01/25 19:55:55] INFO     Found credentials from IAM Role:                                   ]8;id=800161;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=500397;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Using region: us-east-1
USing role: arn:aws:iam::221082214706:role/MYLabRole


In [4]:
# Note: https://github.com/aws/deep-learning-containers/blob/master/available_images.md#hugging-face-training-containers
# If you go to the link, and check huggingface tgraining containers, all require GPU, so set access to GPU use

In [5]:
# HuggingFace DLC for embeddings instead of pip install like data_preprocessing
huggingface_image_uri = (
    "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training:2.1.0-transformers4.36.0-gpu-py310-cu121-ubuntu20.04"
)

In [6]:
# Rest should be similar to preprocessing script

In [7]:
embedding_processor = ScriptProcessor(
    image_uri=huggingface_image_uri,
    command=["python3"],
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.2xlarge",  # I set permissions, can probably run on reduced model size
    volume_size_in_gb=30,
    max_runtime_in_seconds=20500,
    sagemaker_session=session
)

In [8]:
# Run embedding job in .py file containing script logic
embedding_processor.run(
    code="embedding_script.py",
    inputs=[
        ProcessingInput(
            source=f"s3://{bucket_name}/{prefix_in}",
            destination="/opt/ml/processing/input"
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=f"s3://{bucket_name}/{prefix_out}"
        )
    ],
    arguments=[
        "--input-csv-dir", "/opt/ml/processing/input",
        "--output-csv-dir", "/opt/ml/processing/output"
    ]
)

[02/01/25 19:56:07] INFO     Creating processing-job with name                                      ]8;id=101545;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=747186;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1575\1575]8;;\
                             huggingface-pytorch-training-2025-02-01-19-56-07-321                                  

............................/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
CSV chunk files: ['arxiv_preprocessed_part8.csv', 'arxiv_preprocessed_part12.csv', 'arxiv_preprocessed_part13.csv', 'arxiv_preprocessed_part4.csv', 'arxiv_preprocessed_part2.csv', 'arxiv_preprocessed_part6.csv', 'arxiv_preprocessed_part11.csv', 'arxiv_preprocessed_part0.csv', 'arxiv_preprocessed_part5.csv', 'arxiv_preprocessed_part7.csv', 'arxiv_preprocessed_part3.csv', 'arxiv_preprocessed_part1.csv',

In [9]:
# Double check
job_desc = embedding_processor.jobs[-1].describe()
job_desc

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://arxiv-project-bucket/processed-data/',
    'LocalPath': '/opt/ml/processing/input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-221082214706/huggingface-pytorch-training-2025-02-01-19-56-07-321/input/code/embedding_script.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'output-1',
    'S3Output': {'S3Uri': 's3://arxiv-project-bucket/processed-data/embeddings/',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'hugg

 - Should result in CSV with an additional column abstract_embedding, at s3://my-arxiv-project-bucket/processed-data/embeddings/embeddings.csv.